In [ ]:
import re
import os
import csv
import requests
import pandas as pd
from tqdm import tqdm
from bs4 import BeautifulSoup

file_categories = 'Categories_2.csv'
file_parts = 'Parts_2.csv'
file_colors = 'Colors_2.csv'

In [ ]:
force = False
if force:
    for file in [file_categories, file_parts, file_colors]:
        try:
            os.remove(file)
        except:
            pass

In [ ]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}

In [ ]:
df = pd.read_csv(file_categories, header=None)
df.columns = ['Cat_Num', 'Cat_Name', 'Parts']
df = df.set_index('Cat_Num')
df.head(1)

In [ ]:
url = 'https://www.bricklink.com'
bs = BeautifulSoup(requests.get(url+'/catalogTree.asp?itemType=P&itemBrand=1000', headers=headers).text, 'lxml')

cat = re.compile('catString=(\d+)')

data = []
for link in bs.find('table', {'class':'bg-color--white catalog-list__category-list--internal catalog-tree__category-list--internal'}).findAll('a'):
    if 'href' in link.attrs:
        category_name = link.get_text()
        category_number = int(re.search(cat, link['href'])[1])
        category_elements = int(link.next_element.findNext('span').get_text()[1:-1])
        with open(file_categories, 'a') as f:
            writer = csv.writer(f)
            writer.writerow([category_number, category_name, category_elements])

In [ ]:
url = 'https://www.bricklink.com/catalogList.asp?v=0&pg={}&catString={}&itemBrand=1000&catType=P&v=1&viewPrint=Y'
num = re.compile('\?P=(.+)')

with open(file_categories, 'r') as f:
    reader = csv.reader(f)
    for category_number, category_name, category_elements in tqdm(reader):
        pages = -(-int(category_elements)//50)
        for page in range(1, pages+1):
            bs = BeautifulSoup(requests.get(url.format(page, category_number), headers=headers).text, 'lxml')

            for tr in bs.find('table', {'class':'bg-color--white catalog-list__body-main catalog-list__body-main--alternate-row'}).findAll('tr'):
                try:
                    number = re.search(num, tr.find('a')['href'])[1]
                    name = tr.find('strong').get_text()
                    with open(file_parts, 'a') as g:
                        writer = csv.writer(g)
                        writer.writerow([category_number, category_name, number, name])
                except:
                    pass

In [ ]:
df = pd.read_csv(file_parts, header=None)
df.columns = ['Cat_Num', 'Cat_Name', 'Part_Num', 'Part_Name']
print(df.shape)
df.head(1)

In [ ]:
def scrape(cat_num, cat_name, part_num, part_name):
    bs = BeautifulSoup(requests.get(url.format(part_num), headers=headers).text, 'lxml')
    td = bs.find('table', {'class':'pciColorInfoTable'}).findAll('td')
    
    sold_dict = {}
    for span in td[2].findAll('span'):
        try:
            color = span.find('a')['href'].split('colorID=')[-1].split('&')[0]
            sold = span.get_text().rsplit('(', 1)[-1].strip()[:-1]
            sold_dict[color] =  sold
        except:
            pass

    for span in td[3].findAll('span'):
        try:
            color = span.find('a')['href'].split('colorID=')[-1].split('&')[0]
            name = span.find('a').get_text().strip()
            sets = span.get_text().rsplit('(', 1)[-1].strip()[:-1]
            sold = sold_dict[color] if color in sold_dict.keys() else 0

            return [cat_num, cat_name, part_num, part_name, color, name, sets, sold]
        except:
            pass

In [ ]:
if file_colors in os.listdir():
    df_2 = pd.read_csv(file_colors, header=None)
    df_2.columns = ['Cat_Num', 'Cat_Name', 'Part_Num', 'Part_Name', 'Color_Num', 'Color_Name', 'Num_Sets', 'Num_Sold']
else:
    df_2 = pd.DataFrame(columns = ['Cat_Num', 'Cat_Name', 'Part_Num', 'Part_Name', 'Color_Num', 'Color_Name', 'Num_Sets', 'Num_Sold'])
df_2 = df_2.set_index('Part_Num')
print(df_2.shape)
df_2.head(1)

In [ ]:
url = 'https://www.bricklink.com/v2/catalog/catalogitem.page?P={}#T=C'

from multiprocessing.dummy import Pool
import csv

pool = Pool(10)
futures = []
for cat_num, cat_name, part_num, part_name in tqdm(df.values):
    if part_num not in df_2.index:
        futures.append(pool.apply_async(scrape, [cat_num, cat_name, part_num, part_name]))
for ans in tqdm(futures):
    with open('Colors_2.csv', 'a') as f:
        writer = csv.writer(f)
        data = ans.get()
        if data:
            writer.writerow(data)

In [ ]:
df = pd.read_csv(file_colors, header=None)
df.columns = ['Cat_Num', 'Cat_Name', 'Part_Num', 'Part_Name', 'Col_Num', 'Col_Name', 'Num_Sets', 'Num_Sold']
print(df.shape)
df.head(10)